In [1]:
from UCLSE.environment import Market_session, yamlLoad
from UCLSE.test.utils import identical_replay_vars,side_by_side_period_by_period_difference_checker
import copy

In [2]:
import os
pa=os.getcwd()
config_name='config\\mkt_cfg.yml'
config_path=os.path.join(pa,config_name)
environ_dic=yamlLoad(config_path)

In [3]:
#just updates the tradefile every time there is a trade
environ_dic['dump_each_trade']=True
environ_dic['verbose']=False
environ_dic['end_time']=50

Recreating a market session is vital for refactoring and debugging. Moreover setting the random seed on initialization didn't work when I tried to recreate results. Because the exchange and the traders repeatedly make random actions, recreation is not straightforward.

After extensive rewriting, the simulate method is now separated into a simulate_one_period method which is then repeated for the duration of the experiment. 

simulate_one_period does exactly what you would expect. Within this method, I have separated out the individual tasks that a simulation period undergoes: 

1. Getting supply and demand for the traders
2. Cancel any overwritten orders (still in a one order world)
3. Pick a  limit order from a trader chosen at random
4. Send that order to the exchange for processing.
        This might result in a trade being printed
5. Publish the LOB for traders to update their records
6. (Optional) Record the happenings of the period for recreation 


The simulate method now has the option of recording the events of session stored in attribute  Market_session.replay_vars, so that the session can be identically recreated.

The replay_vars attribute is a dictionary which can be entered as an input into the simulate method for recreation.

Begin by instantiating a Market_session and then copying it exactly.

In [4]:
import copy
sess=Market_session(**environ_dic)
sess1=copy.deepcopy(sess)

Then we can run the first simulation, store the events and push those events into the second simulation session to make sure that the end result is the same.

In [5]:
sess.simulate(sess.trade_stats_df3,recording=True)
sess1.simulate(sess1.trade_stats_df3,replay_vars=sess.replay_vars,recording=True)
(sess1.df.fillna(0)==sess.df.fillna(0)).all() #note that the best_bid or best_bid do not always give a True value. Because nan!=nan.

avg_balance.csv
avg_balance.csv


expid                    True
time                     True
ZIC       balance_sum    True
          n              True
          pc             True
ZIP       balance_sum    True
          n              True
          pc             True
SHVR      balance_sum    True
          n              True
          pc             True
GVWY      balance_sum    True
          n              True
          pc             True
best_bid                 True
best_ask                 True
dtype: bool

This function checks that the two replay_vars attributes are the same and returns the difference if it exists:
    

In [6]:
identical_replay_vars(sess,sess1)

no differences found


{}

For a more forensic approach to difference checking, this function will look for differences at the end of every period and stop if one occurs.

In [7]:
sess=Market_session(**environ_dic)
sess1=copy.deepcopy(sess)
side_by_side_period_by_period_difference_checker(sess,sess1)
(sess1.df==sess.df).all()

avg_balance.csv
avg_balance.csv


expid                     True
time                      True
ZIC       balance_sum     True
          n               True
          pc              True
ZIP       balance_sum     True
          n               True
          pc              True
SHVR      balance_sum     True
          n               True
          pc              True
GVWY      balance_sum     True
          n               True
          pc              True
best_bid                 False
best_ask                 False
dtype: bool

In [13]:
sess1.df

expid     time         ZIC                  SHVR            \
                            balance_sum   n    pc balance_sum   n    pc   
4.0000   trial0001   4.0000         0.0  20  0.00         0.0  20  0.00   
6.7500   trial0001   6.7500         0.0  20  0.00        10.0  20  0.50   
7.4625   trial0001   7.4625         0.0  20  0.00        20.0  20  1.00   
7.6500   trial0001   7.6500         0.0  20  0.00        30.0  20  1.50   
7.7250   trial0001   7.7250         0.0  20  0.00        30.0  20  1.50   
8.2875   trial0001   8.2875         0.0  20  0.00        30.0  20  1.50   
10.1625  trial0001  10.1625         0.0  20  0.00        37.0  20  1.85   
10.9000  trial0001  10.9000         0.0  20  0.00        37.0  20  1.85   
12.3375  trial0001  12.3375        11.0  20  0.55        37.0  20  1.85   
14.1000  trial0001  14.1000        11.0  20  0.55        48.0  20  2.40   
14.9875  trial0001  14.9875        11.0  20  0.55        58.0  20  2.90   
15.9000  trial0001  15.9000        11.0  20  0.55        68.0  20  3.40   
16.2500  trial0001  16.2500        19.0  20  0.95        68.0  20  3.40   
18.5000  trial0001  18.5000        28.0  20  1.40        69.0  20  3.45   
19.6625  trial0001  19.6625        28.0  20  1.40        69.0  20  3.45   
20.0125  trial0001  20.0125        28.0  20  1.40        69.0  20  3.45   
21.0125  trial0001  21.0125        28.0  20  1.40        69.0  20  3.45   
22.6875  trial0001  22.6875        38.0  20  1.90        69.0  20  3.45   
23.9625  trial0001  23.9625        38.0  20  1.90        69.0  20  3.45   
24.7125  trial0001  24.7125        38.0  20  1.90        69.0  20  3.45   
26.3375  trial0001  26.3375        38.0  20  1.90        80.0  20  4.00   
28.1625  trial0001  28.1625        38.0  20  1.90        80.0  20  4.00   
28.5625  trial0001  28.5625        38.0  20  1.90        90.0  20  4.50   
28.6375  trial0001  28.6375        38.0  20  1.90        90.0  20  4.50   
28.9500  trial0001  28.9500        47.0  20  2.35        90.0  20  4.50   
29.0625  trial0001  29.0625        47.0  20  2.35        90.0  20  4.50   
30.7125  trial0001  30.7125        47.0  20  2.35        91.0  20  4.55   
30.9000  trial0001  30.9000        47.0  20  2.35       101.0  20  5.05   
32.3250  trial0001  32.3250        47.0  20  2.35       101.0  20  5.05   
33.6250  trial0001  33.6250        47.0  20  2.35       101.0  20  5.05   
34.4125  trial0001  34.4125        47.0  20  2.35       110.0  20  5.50   
35.0500  trial0001  35.0500        47.0  20  2.35       119.0  20  5.95   
36.0875  trial0001  36.0875        47.0  20  2.35       119.0  20  5.95   
36.5625  trial0001  36.5625        47.0  20  2.35       119.0  20  5.95   
37.2000  trial0001  37.2000        47.0  20  2.35       120.0  20  6.00   
37.5750  trial0001  37.5750        47.0  20  2.35       120.0  20  6.00   
37.5875  trial0001  37.5875        55.0  20  2.75       120.0  20  6.00   
41.6875  trial0001  41.6875        55.0  20  2.75       130.0  20  6.50   
42.1875  trial0001  42.1875        55.0  20  2.75       140.0  20  7.00   
42.4000  trial0001  42.4000        55.0  20  2.75       140.0  20  7.00   
42.9000  trial0001  42.9000        55.0  20  2.75       148.0  20  7.40   
43.2250  trial0001  43.2250        55.0  20  2.75       148.0  20  7.40   
43.8625  trial0001  43.8625        55.0  20  2.75       148.0  20  7.40   
43.9000  trial0001  43.9000        55.0  20  2.75       150.0  20  7.50   
49.2375  trial0001  49.2375        55.0  20  2.75       161.0  20  8.05   
49.2625  trial0001  49.2625        55.0  20  2.75       161.0  20  8.05   
49.6500  trial0001  49.6500        55.0  20  2.75       161.0  20  8.05   
50.0000  trial0001  50.0000        55.0  20  2.75       161.0  20  8.05   

               GVWY                   ZIP           best_bid best_ask  
        balance_sum   n    pc balance_sum   n    pc                    
4.0000         10.0  20  0.50         0.0  20  0.00      105      117  
6.7500         10.0  20  0.50         0.0  20 

In [14]:
sess.df

expid     time         ZIC                  SHVR            \
                            balance_sum   n    pc balance_sum   n    pc   
4.0000   trial0001   4.0000         0.0  20  0.00         0.0  20  0.00   
6.7500   trial0001   6.7500         0.0  20  0.00        10.0  20  0.50   
7.4625   trial0001   7.4625         0.0  20  0.00        20.0  20  1.00   
7.6500   trial0001   7.6500         0.0  20  0.00        30.0  20  1.50   
7.7250   trial0001   7.7250         0.0  20  0.00        30.0  20  1.50   
8.2875   trial0001   8.2875         0.0  20  0.00        30.0  20  1.50   
10.1625  trial0001  10.1625         0.0  20  0.00        37.0  20  1.85   
10.9000  trial0001  10.9000         0.0  20  0.00        37.0  20  1.85   
12.3375  trial0001  12.3375        11.0  20  0.55        37.0  20  1.85   
14.1000  trial0001  14.1000        11.0  20  0.55        48.0  20  2.40   
14.9875  trial0001  14.9875        11.0  20  0.55        58.0  20  2.90   
15.9000  trial0001  15.9000        11.0  20  0.55        68.0  20  3.40   
16.2500  trial0001  16.2500        19.0  20  0.95        68.0  20  3.40   
18.5000  trial0001  18.5000        28.0  20  1.40        69.0  20  3.45   
19.6625  trial0001  19.6625        28.0  20  1.40        69.0  20  3.45   
20.0125  trial0001  20.0125        28.0  20  1.40        69.0  20  3.45   
21.0125  trial0001  21.0125        28.0  20  1.40        69.0  20  3.45   
22.6875  trial0001  22.6875        38.0  20  1.90        69.0  20  3.45   
23.9625  trial0001  23.9625        38.0  20  1.90        69.0  20  3.45   
24.7125  trial0001  24.7125        38.0  20  1.90        69.0  20  3.45   
26.3375  trial0001  26.3375        38.0  20  1.90        80.0  20  4.00   
28.1625  trial0001  28.1625        38.0  20  1.90        80.0  20  4.00   
28.5625  trial0001  28.5625        38.0  20  1.90        90.0  20  4.50   
28.6375  trial0001  28.6375        38.0  20  1.90        90.0  20  4.50   
28.9500  trial0001  28.9500        47.0  20  2.35        90.0  20  4.50   
29.0625  trial0001  29.0625        47.0  20  2.35        90.0  20  4.50   
30.7125  trial0001  30.7125        47.0  20  2.35        91.0  20  4.55   
30.9000  trial0001  30.9000        47.0  20  2.35       101.0  20  5.05   
32.3250  trial0001  32.3250        47.0  20  2.35       101.0  20  5.05   
33.6250  trial0001  33.6250        47.0  20  2.35       101.0  20  5.05   
34.4125  trial0001  34.4125        47.0  20  2.35       110.0  20  5.50   
35.0500  trial0001  35.0500        47.0  20  2.35       119.0  20  5.95   
36.0875  trial0001  36.0875        47.0  20  2.35       119.0  20  5.95   
36.5625  trial0001  36.5625        47.0  20  2.35       119.0  20  5.95   
37.2000  trial0001  37.2000        47.0  20  2.35       120.0  20  6.00   
37.5750  trial0001  37.5750        47.0  20  2.35       120.0  20  6.00   
37.5875  trial0001  37.5875        55.0  20  2.75       120.0  20  6.00   
41.6875  trial0001  41.6875        55.0  20  2.75       130.0  20  6.50   
42.1875  trial0001  42.1875        55.0  20  2.75       140.0  20  7.00   
42.4000  trial0001  42.4000        55.0  20  2.75       140.0  20  7.00   
42.9000  trial0001  42.9000        55.0  20  2.75       148.0  20  7.40   
43.2250  trial0001  43.2250        55.0  20  2.75       148.0  20  7.40   
43.8625  trial0001  43.8625        55.0  20  2.75       148.0  20  7.40   
43.9000  trial0001  43.9000        55.0  20  2.75       150.0  20  7.50   
49.2375  trial0001  49.2375        55.0  20  2.75       161.0  20  8.05   
49.2625  trial0001  49.2625        55.0  20  2.75       161.0  20  8.05   
49.6500  trial0001  49.6500        55.0  20  2.75       161.0  20  8.05   
50.0000  trial0001  50.0000        55.0  20  2.75       161.0  20  8.05   

               GVWY                   ZIP           best_bid best_ask  
        balance_sum   n    pc balance_sum   n    pc                    
4.0000         10.0  20  0.50         0.0  20  0.00      105      117  
6.7500         10.0  20  0.50         0.0  20 